In [1]:
import pandas as pd
from inflection import underscore, parameterize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import accuracy_score, classification_report
from statsmodels.stats.descriptivestats import describe, Description

In [2]:
df = pd.read_csv('data/winequality-red.csv', sep=';', low_memory=False)
df.rename(columns={x: underscore(parameterize(x)) for x in df.columns}, inplace=True)
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
describe(df, stats=['nobs', 'missing', 'mean', 'median','mode','std','iqr','skew',
                    'kurtosis', 'min', 'max', 'coef_var','distinct'], percentiles=['25','50','75'])

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
nobs,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
missing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
iqr,2.100000,0.250000,0.330000,0.700000,0.020000,14.000000,40.000000,0.002235,0.190000,0.180000,1.600000,1.000000
coef_var,0.209276,0.339244,0.718888,0.555351,0.538095,0.658911,0.707917,0.001893,0.046627,0.257551,0.102242,0.143287
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
skew,0.981829,0.670962,0.318039,4.536395,5.675017,1.249394,1.514109,0.071221,0.193502,2.426393,0.860021,0.217597
kurtosis,4.124856,4.217963,2.209717,31.524438,44.581708,5.013490,6.794172,3.927411,3.800671,14.679884,3.195654,3.292031


In [6]:
df.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
ph                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [3]:
df.isnull().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
ph                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [14]:
df.corr(method='pearson')

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
fixed_acidity,1.000000,-0.256131,0.671703,0.114777,0.093705,-0.153794,-0.113181,0.668047,-0.682978,0.183006,-0.061668,0.124052
volatile_acidity,-0.256131,1.000000,-0.552496,0.001918,0.061298,-0.010504,0.076470,0.022026,0.234937,-0.260987,-0.202288,-0.390558
citric_acid,0.671703,-0.552496,1.000000,0.143577,0.203823,-0.060978,0.035533,0.364947,-0.541904,0.312770,0.109903,0.226373
residual_sugar,0.114777,0.001918,0.143577,1.000000,0.055610,0.187049,0.203028,0.355283,-0.085652,0.005527,0.042075,0.013732
chlorides,0.093705,0.061298,0.203823,0.055610,1.000000,0.005562,0.047400,0.200632,-0.265026,0.371260,-0.221141,-0.128907
free_sulfur_dioxide,-0.153794,-0.010504,-0.060978,0.187049,0.005562,1.000000,0.667666,-0.021946,0.070377,0.051658,-0.069408,-0.050656
total_sulfur_dioxide,-0.113181,0.076470,0.035533,0.203028,0.047400,0.667666,1.000000,0.071269,-0.066495,0.042947,-0.205654,-0.185100
density,0.668047,0.022026,0.364947,0.355283,0.200632,-0.021946,0.071269,1.000000,-0.341699,0.148506,-0.496180,-0.174919
ph,-0.682978,0.234937,-0.541904,-0.085652,-0.265026,0.070377,-0.066495,-0.341699,1.000000,-0.196648,0.205633,-0.057731
sulphates,0.183006,-0.260987,0.312770,0.005527,0.371260,0.051658,0.042947,0.148506,-0.196648,1.000000,0.093595,0.251397


In [16]:
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [4]:
df.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [8]:
X_df = df.iloc[:,:11]
y_df = df['quality']

scaler = MinMaxScaler()
df_scale = scaler.fit_transform(X_df)

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=1)

In [12]:
df_scale.min()

0.0

In [17]:
clf_KNN = KNeighborsClassifier(n_neighbors=5)
clf_KNN.fit(X_train, y_train)
previsoes = clf_KNN.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

# cm = ConfusionMatrix(clf_KNN)
# cm.fit(X_train, y_train)
# cm.score(X_test, y_test)

# print(classification_report(y_test, previsoes))

50.416666666666664


In [7]:
clf_arvore = DecisionTreeClassifier()
clf_arvore.fit(X_train, y_train)
previsoes = clf_arvore.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

# cm = ConfusionMatrix(clf_arvore)
# cm.fit(X_train, y_train)
# cm.score(X_test, y_test)

# print(classification_report(y_test, previsoes))

60.20833333333333


In [8]:
clf_floresta = RandomForestClassifier(max_depth=10, random_state=1)
clf_floresta.fit(X_train, y_train)
previsoes = clf_floresta.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

# cm = ConfusionMatrix(clf_floresta)
# cm.fit(X_train, y_train)
# cm.score(X_test, y_test)

# print(classification_report(y_test, previsoes))

68.125


In [34]:
clf_svm=SVC(gamma='auto',kernel='rbf')
clf_svm.fit(X_train, y_train)
previsoes = clf_svm.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

# cm = ConfusionMatrix(clf_svm)
# cm.fit(X_train, y_train)
# cm.score(X_test, y_test)

# print(classification_report(y_test, previsoes))

56.458333333333336


In [33]:
clf_mlp = MLPClassifier( alpha=1e-5, hidden_layer_sizes=(5,5), random_state=1)
clf_mlp.fit(X_train, y_train)
previsoes = clf_mlp.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

# cm = ConfusionMatrix(clf_mlp)
# cm.fit(X_train, y_train)
# cm.score(X_test, y_test)

# print(classification_report(y_test, previsoes))

48.75


/home/machadom729/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
df['qualit'] = df['quality'].apply(lambda x: 'bom' if x>5 else 'ruim')

X_df = df.iloc[:,:11]
y_df = df.iloc[:,12]

scaler = MinMaxScaler()
df_scale = scaler.fit_transform(X_df)

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=1)

clf_mlp = MLPClassifier( alpha=1e-5, hidden_layer_sizes=(5,5), random_state=1)
clf_mlp.fit(X_train, y_train)
previsoes = clf_mlp.predict(X_test)

print(accuracy_score(y_test, previsoes) * 100)

61.458333333333336


/home/machadom729/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
